In [1]:
import numpy as np
import xarray as xr
from main import paths, idx_1d, LAT_DEG, lx

# Path to save figures, save data and OFAM model output.
fpath, dpath, xpath = paths()

In [30]:
# Longitudes to integrate velocity.
# 143E, 156E, 165E, 170E, 180E, 170W, 155W, 140W, 125W, 110W, 95W
lons = [180]#[143, 156, 165, 170, 180, 190, 205, 220, 235, 250, 265]
# Depth integration boundaries.
# NOTE: slice will find the closest, but probably smallest depth level.
levs = [100, 300]
# latitude integration boundaries. 
# NOTE must add 0.1 deg when slicing.
lats = [-0.5, 0.5]
# Climatology year range.
year = [1981, 2012]

# Open dataset containing the monthly mean climatololgy (see climo.py).
clim = xr.open_dataset(xpath.joinpath('ocean_u_{}-{}_climo.nc'.format(*year)))

# Slice dataset (so integration bounds do not need to be explicitly supplied).
ds = clim.sel(st_ocean=slice(levs[0], levs[-1]), 
              yu_ocean=slice(lats[0], lats[-1] + 0.1), 
              xu_ocean=lons)

# Find indexes of each depth level for depth bounds. 
# NOTE must calculate on unsliced dataset.
k0 = idx_1d(clim.st_ocean, levs[0])
k1 = idx_1d(clim.st_ocean, levs[-1])

# Slice st_ocean_edges (used to calculate width of each grid cell 'dk').
ds = ds.isel(st_edges_ocean=slice(k0+1, k1+2))

# Width of grid cell [m].
dy = 0.1 * LAT_DEG

# Depth of each grid cell [m].
dk = np.array([clim.st_edges_ocean[i+1].item() - clim.st_edges_ocean[i].item() 
               for i in range(len(ds.st_edges_ocean))])

# Depth levels multiplied by grid width.
dkdy = dk*dy

# Array to save transport.
uvo = np.zeros(ds.u.shape)

In [5]:
# Calculate transport.
for i in range(len(ds.xu_ocean)):
    for j in range(len(ds.yu_ocean)):
        for t in range(12):
            for k in range(len(ds.st_ocean)):
                if ds.u[t, k, j, i] >= 0:
                    # Velocity x Depth x Width (if travelling eastward).
                    uvo[t, k, j, i] = ds.u[t, k, j, i]*dkdy[k]

In [6]:
# Create new dataset with the same coordinates as original files.
df = xr.Dataset(coords = ds.coords)
# Save transport file.
df['uvo'] = (('month', 'st_ocean', 'yu_ocean', 'xu_ocean'), uvo)
# Add some attributes.
df.uvo.attrs['long_name'] = 'Initial zonal volume transport'
df.uvo.attrs['units'] = 'm3/sec'
df.uvo.attrs['standard_name'] = 'sea_water_x_transport'
df.uvo.attrs['description'] = 'Monthly mean transport ({}-{}).'.format(*year)


# # Save dataset to netcdf.
# df.to_netcdf(dpath.joinpath('EUC_transport_grid_{}-{}.nc'.format(*year)))

# clim.close()
# ds.close()
# df.close()
df.uvo.sel(st_ocean=slice(250, 300), month=1)

In [12]:
df.uvo.sel(st_ocean=slice(250, 300), month=1)

<xarray.DataArray 'uvo' (st_ocean: 2, yu_ocean: 11, xu_ocean: 1)>
array([[[21806.847635],
        [23316.55028 ],
        [24911.363776],
        [26330.828966],
        [27277.838226],
        [27651.99473 ],
        [27545.094989],
        [27220.703094],
        [26959.188655],
        [26771.293833],
        [26331.983029]],

       [[14663.582468],
        [15684.729082],
        [16712.643941],
        [17642.28744 ],
        [18374.472169],
        [18852.900431],
        [19053.591747],
        [19027.33267 ],
        [18905.011634],
        [18727.995994],
        [18409.405737]]])
Coordinates:
  * st_ocean  (st_ocean) float64 255.9 286.6
  * xu_ocean  (xu_ocean) float64 180.0
  * yu_ocean  (yu_ocean) float64 -0.5 -0.4 -0.3 -0.2 -0.1 ... 0.2 0.3 0.4 0.5
    month     int64 1
Attributes:
    long_name:      Initial zonal volume transport
    units:          m3/sec
    standard_name:  sea_water_x_transport
    description:    Monthly mean transport (1981-2012).

In [ ]:
df.uvo.attrs['long_name'] = 'Initial zonal volume transport'

In [ ]:
df.uvo.mean('month')

In [18]:
print(df.uvo.shape)
np.append(2, df.uvo.shape)


(12, 15, 11, 1)


array([ 2, 12, 15, 11,  1])

In [ ]:
ds.st_edges_ocean

In [ ]:
df = xr.open_dataset(dpath.joinpath('EUC_transport_grid_{}-{}.nc'.format(*year)))

In [26]:
from main import lx

In [28]:
lx['exp']

['Historical', 'RCP85', 'Change']

In [4]:
years = lx['years']
years[0][0]

1981

In [ ]:
np.arange(145, 270, 5)

In [ ]:
# files = []
# for t in range(13):
#     files.append()
d = xr.open_dataset(xpath.joinpath('ocean_u_1979_01.nc'))
d

In [19]:
mm = xr.open_dataset(dpath.joinpath('ocean_mesh_mask.nc'))

In [21]:
tmp = mm.coords
tmp

Coordinates:
  * nv              (nv) float64 1.0 2.0
  * st_edges_ocean  (st_edges_ocean) float64 0.0 5.0 10.0 ... 4.056e+03 5e+03
  * st_ocean        (st_ocean) float64 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * xu_ocean        (xu_ocean) float64 120.0 120.1 120.2 ... 294.7 294.8 294.9
  * yu_ocean        (yu_ocean) float64 -15.0 -14.9 -14.8 ... 14.7 14.8 14.9
  * xt_ocean        (xt_ocean) float64 120.1 120.2 120.2 ... 294.8 294.9 295.0
  * yt_ocean        (yt_ocean) float64 -14.95 -14.85 -14.75 ... 14.85 14.95

In [ ]:
df.month.rename('Time')
df['Time'] = pd.date_range(start='1979-01-01', periods=12, freq='M')
df

In [60]:
dx = ds.mean('month').isel(yu_ocean=5)
for i in range(15):
    print('{:.01f}, {:.03f}'.format(dx.st_ocean[i].item(), dx.u[i].item()))

105.0, -0.002
115.0, 0.056
125.0, 0.125
135.0, 0.196
145.0, 0.262
155.0, 0.318
165.0, 0.356
175.0, 0.377
185.0, 0.385
195.0, 0.380
205.2, 0.366
217.1, 0.343
233.2, 0.303
255.9, 0.240
286.6, 0.153


In [4]:
from pathlib import Path
home = Path('/g', 'data', 'e14', 'as3189', 'OFAM')
tpath = home.joinpath('TAU', 'adcp_xyzt_mon.ascii')

In [15]:

from astropy.io import ascii

ModuleNotFoundError: No module named 'astropy'

In [17]:
ds = np.loadtxt(tpath)

ValueError: could not convert string to float: 'GTMBA'

In [10]:
ds.read()

' GTMBA Project Office, Dr. Michael J. McPhaden, director.  Sep 18 2019\n Acoustic Doppler Current Profiles, Units: cm/sec, Missing = -999.9\n Oceanographic Convention: (1,1) is NE at sqrt(2) cm/s\n nlon =  7, nlat =  2, ndep = 84, ntim =  381\n Lon = 147 156 165 190 220 235 250\n Lat =   0   8\n Depth =           0     10     15     20     25     30     35     40\n                  45     50     55     60     65     70     75     80\n                  85     90     95    100    105    110    115    120\n                 125    130    135    140    145    150    155    160\n                 165    170    175    180    185    190    195    200\n                 205    210    215    220    225    230    235    240\n                 245    250    255    260    265    270    275    280\n                 285    290    295    300    305    310    315    320\n                 325    330    335    340    345    350    355    360\n                 365    370    375    380    385    390    395  